# Tactical Asset Allocation

In [1]:
%%capture
%reload_ext autoreload
%autoreload 2
%cd ..
%cd src

In [16]:
from pytaa.tools.data import get_strategy_price_data
from pytaa.strategy.static import STRATEGIES
from pytaa.strategy.strategies import StrategyPipeline
from pytaa.strategy.signals import Signal
from pytaa.backtest.positions import (
    EqualWeights, RiskParity, StaticAllocation, vigilant_allocation, kipnis_allocation, aqr_trend_allocation, protective_allocation, haa_allocation, best_mix_between_weights
)
from pytaa.backtest.performance import Tearsheet
from pytaa.backtest.returns import Backtester

import pandas as pd
import numpy as np
from functools import reduce


In [4]:
start, end = "2011-01-01", "2025-09-14"
rebalance_dates = pd.bdate_range(start, end, freq="BME")

# compute the next business month-end after 'end' and append if it's beyond the last date
next_bme = pd.to_datetime(end) + pd.offsets.BMonthEnd(1)
if next_bme > rebalance_dates.max():
    rebalance_dates = rebalance_dates.union(pd.DatetimeIndex([next_bme]))

pipeline = StrategyPipeline(STRATEGIES)
data = get_strategy_price_data(pipeline, start, end).dropna()


In [6]:
pipeline = StrategyPipeline(STRATEGIES)
data = get_strategy_price_data(pipeline, start, end).dropna()

In [103]:
all_strategies = []

vt = EqualWeights(['VT'], rebalance_dates, "vt").weights
spy = EqualWeights(['SPY'], rebalance_dates, "spy").weights
all_strategies.extend([vt,spy])

# ivy
strategy = pipeline.ivy
strategy_data = data[strategy.risk_assets]
signals = Signal(strategy_data).sma_crossover(10).dropna()

weights = signals.apply(lambda x: np.where(x > 0, strategy.weights[0], 0))
weights.loc[:, strategy.safe_assets] = 1 - weights.sum(axis=1)
weights.columns.name = "ID"
all_strategies.append(weights.stack().rename(strategy.tag).to_frame())

# benchmark strategies
assets = ["SPY", "BND", "BIL", "GLD"]
returns = data.pct_change().dropna().loc[:, assets]

ew = EqualWeights(assets, rebalance_dates).weights
rp = RiskParity(assets, rebalance_dates, returns).weights
all_strategies.append([ew, rp])

sixty_forty = StaticAllocation(['SPY', 'TLT'], rebalance_dates, {'SPY': 0.6, 'TLT': 0.4}, name='60_40').weights
all_strategies.append([sixty_forty])

# robust asset allocation
strategy = pipeline.raab
strategy_data = data[strategy.get_tickers()]
signal_1 = Signal(strategy_data).classic_momentum(end=0).dropna()
signal_2 = Signal(strategy_data).sma_crossover(12, False).dropna().loc[:, strategy.risk_assets]
cond_1 = np.where(signal_1[strategy.risk_assets].gt(signal_1[strategy.safe_assets].values), 1, 0)
cond_2 = np.where(strategy_data[strategy.risk_assets].reindex(signal_2.index) > signal_2, 1, 0)
final_signal = cond_1 + cond_2
cash = np.atleast_2d(len(strategy.risk_assets) - np.sum(final_signal / 2, 1)).T
idx, cols = signal_1.index, signal_1.columns
weights = pd.DataFrame(np.hstack([final_signal, cash]), index=idx, columns=cols)
weights = weights.div(weights.sum(axis=1).values.reshape(-1, 1))
weights.columns.name = "ID"
all_strategies.append(weights.stack().rename(strategy.tag).to_frame())

# vigilant asset allocation g12
strategy = pipeline.vaag12
strategy_data = data[strategy.get_tickers()]
signal = Signal(strategy_data).momentum_score()
args = (strategy.risk_assets, strategy.safe_assets)
weights = pd.concat([x for x in signal.apply(lambda x: vigilant_allocation(x, *args), axis=1)])
weights.index = signal.index
weights.columns.name = "ID"
all_strategies.append(weights.stack().rename(strategy.tag).to_frame())

# vigilant asset allocation g4
strategy = pipeline.vaag4
strategy_data = data[strategy.get_tickers()]
signal = Signal(strategy_data).momentum_score()
args = (strategy.risk_assets, strategy.safe_assets, 1, 1)
weights = pd.concat([x for x in signal.apply(lambda x: vigilant_allocation(x, *args), axis=1)])
weights.index = signal.index
weights.columns.name = "ID"
all_strategies.append(weights.stack().rename(strategy.tag).to_frame())

# gem dual momentum
strategy = pipeline.gdm
strategy_data = data[strategy.get_tickers()]
signal = Signal(strategy_data).classic_momentum(start=12, end=0)
cond_1, cond_2 = signal["IVV"].ge(signal["BIL"]), signal["IVV"].ge(signal["VEU"])
spy = np.where(cond_1 & cond_2, 1, 0).reshape(-1,1)
efa = np.where(cond_1 & ~cond_2, 1, 0 ).reshape(-1,1)
agg = np.where(~cond_1, 1, 0).reshape(-1,1)
weights = pd.DataFrame(np.hstack([spy, efa, agg]), index=signal.index, columns=["IVV", "VEU", "BND"])
weights.columns.name = "ID"
all_strategies.append(weights.stack().rename(strategy.tag).to_frame())

# diversified gem dual momentum
strategy = pipeline.dgdm
strategy_data = data[strategy.get_tickers()]
mom_periods = [6, 7, 8, 9, 10, 11, 12]
monthly = strategy_data.resample("BME").last()
spy, agg = np.zeros((monthly.shape[0], 1)), np.zeros((monthly.shape[0], 1))
efa = np.zeros((monthly.shape[0], 1))

for w in mom_periods:
    signal = Signal(strategy_data).classic_momentum(start=w, end=0)
    cond_1, cond_2 = signal["SPY"].ge(signal["AGG"]), signal["SPY"].ge(signal["EFA"])
    spy += np.where(cond_1 & cond_2, 1, 0).reshape(-1,1)
    efa += np.where(cond_1 & ~cond_2, 1, 0 ).reshape(-1,1)
    agg += np.where(~cond_1, 1, 0).reshape(-1,1)
idx, cols = monthly.index, monthly.columns
weights = pd.DataFrame(np.hstack([spy, efa, agg]) / len(mom_periods), index=idx, columns=cols)
weights.columns.name = "ID"
all_strategies.append(weights.stack().rename(strategy.tag).to_frame())

# kipnis defensive asset allocation
strategy = pipeline.kdaaa
strategy_data = data[strategy.get_tickers()]
signal = Signal(strategy_data).momentum_score().dropna()
returns = strategy_data.pct_change()
weights = kipnis_allocation(returns, signal, rebalance_dates, strategy.safe_assets, strategy.risk_assets, strategy.safe_assets)
weights.columns.name, weights.index.name = "ID", "Date"
all_strategies.append(weights.stack().rename(strategy.tag).to_frame())

# trend is our friend
strategy = pipeline.tiof
strategy_data = data[strategy.get_tickers()]
returns = strategy_data.pct_change()
signal = Signal(strategy_data).sma_crossover(lookback=10)
weights = aqr_trend_allocation(returns, signal, rebalance_dates, strategy.risk_assets, "BIL")
all_strategies.append(weights.stack().rename(strategy.tag).to_frame())

# Generalized Protective Asset Allocation
strategy = pipeline.gpm
strategy_data = data[strategy.get_tickers()]
signal = Signal(strategy_data)
score = signal.protective_momentum_score(strategy.risk_assets)
weights = protective_allocation(
    signal.monthly_prices,
    score,
    rebalance_dates,
    strategy.risk_assets,
    strategy.safe_assets,
    2,
    3)
weights.columns.name, weights.index.name = "ID", "Date"
all_strategies.append(weights.stack().rename(strategy.tag).to_frame())

# Hybrid Asset Allocation G8/T4
strategy = pipeline.haa4
strategy_data = data[strategy.get_tickers()]
signal = Signal(strategy_data)
score = signal.average_return()
weights = haa_allocation(
    score,
    rebalance_dates,
    strategy.risk_assets,
    strategy.safe_assets,
    "TIP")
weights.columns.name, weights.index.name = "ID", "Date"
all_strategies.append(weights.stack().rename(strategy.tag).to_frame())

# Hybrid Asset Allocation U1/T1
strategy = pipeline.haa1
strategy_data = data[strategy.get_tickers()]
signal = Signal(strategy_data)
score = signal.average_return()
weights = haa_allocation(
    score,
    rebalance_dates,
    strategy.risk_assets,
    strategy.safe_assets,
    "TIP")
weights.columns.name, weights.index.name = "ID", "Date"
all_strategies.append(weights.stack().rename(strategy.tag).to_frame())

In [104]:
# concat all strats
all_strategies = reduce(lambda x, y: x.join(y, how="outer"), all_strategies).fillna(0)

In [105]:
bt = Backtester(all_strategies, "USD")
port_price_returns = bt.run(return_type="price", prices=data)
port_price_returns

,vt,spy,IVY,EW,RP,60_40,RAAB,VAAG12,VAAG4,GDM,DGDM,KDAAA,TIOF,GPM,HAA4,HAA1
Date,,,,,,,,,,,,,,,,
2013-04-30,0.048437,0.035941,0.000000,0.025139,0.000000,0.021305,0.000000,0.000000,0.000000,0.001546,0.002787,0.000000,0.000000,0.046417,0.000000,0.000000
2013-05-31,-0.005589,0.023610,0.000000,-0.014354,0.000000,-0.012878,0.000000,0.000000,0.000000,-0.019258,-0.020012,0.000000,0.000000,0.032901,0.000000,0.000000
2013-06-28,-0.026714,-0.013345,0.000000,-0.035157,-0.003067,-0.021092,0.000000,0.000000,0.000000,-0.016477,-0.015654,0.000000,0.000000,-0.015159,0.000000,0.000000
2013-07-31,0.051640,0.051677,0.000000,0.032445,0.001413,0.021974,0.000000,0.000000,0.000000,0.003802,0.002692,0.000000,0.000000,0.062720,0.000000,0.000000
2013-08-30,-0.024552,-0.029992,0.000000,0.003368,-0.000847,-0.023357,0.000000,0.000000,0.000000,-0.008582,-0.008264,0.000000,0.000000,-0.021865,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-30,0.058062,0.062845,0.010237,0.014794,0.003538,0.024865,0.031149,-0.012396,-0.012396,0.046359,0.009352,0.007582,0.001207,0.019130,0.022186,0.062845
2025-06-30,0.046732,0.051386,0.022157,0.018488,0.004451,0.041488,0.025982,0.003347,0.003347,0.051826,0.026536,0.016020,0.014484,0.018327,0.039170,0.051386
2025-07-31,0.010971,0.023032,0.008789,0.004451,0.003583,0.009260,0.002346,0.004779,0.006633,-0.009223,-0.020920,-0.006529,0.009965,-0.001923,0.008183,0.023032


In [106]:
import plotly.express as px
import plotly.io as pio
# Use browser renderer to avoid nbformat dependency in this environment
pio.renderers.default = 'browser'

# plot cumulative returns in interactive diagram
df = port_price_returns[port_price_returns.index >= "2014-01-01"].add(1).cumprod()
fig = px.line(df, title="Cumulative Total Returns")
fig.show(renderer='browser')


In [111]:
# list the annual returns of the strategies between 2011 and 2025
annual_returns = port_price_returns.resample("YE").apply(lambda x: (x + 1).prod() - 1)
annual_returns = annual_returns.loc[annual_returns.index >= "2014-01-01"]
annual_returns
#annual_returns.plot.bar(title="Annual Returns of Strategies", figsize=(12, 6));

,vt,spy,IVY,EW,RP,60_40,RAAB,VAAG12,VAAG4,GDM,DGDM,KDAAA,TIOF,GPM,HAA4,HAA1
Date,,,,,,,,,,,,,,,,
2014-12-31,0.036800,0.134638,0.060057,0.043034,0.004104,0.190013,0.000692,0.012734,0.029073,0.137012,0.128000,0.019967,0.055418,0.045200,0.017383,0.074147
2015-12-31,-0.018614,0.012343,-0.037366,-0.019664,-0.002841,0.005805,0.024184,-0.001448,-0.006291,-0.066053,-0.062867,-0.037856,-0.006864,-0.018277,-0.030144,0.016265
2016-12-31,0.084975,0.119979,0.062378,0.060640,0.008209,0.080187,0.141306,0.049600,0.062424,0.068767,0.063774,0.045987,0.035972,0.023040,0.123299,0.158338
2017-12-31,0.244964,0.217054,0.102538,0.096091,0.021603,0.166079,0.141105,0.076481,0.137359,0.208977,0.188443,0.076431,0.046483,0.135991,0.143431,0.164216
2018-12-31,-0.097631,-0.045690,-0.016003,-0.008771,0.014309,-0.029335,-0.062770,0.090725,0.112977,-0.079372,-0.041175,0.085753,-0.004344,0.052864,0.066480,0.116937
2019-12-31,0.268139,0.312239,0.080952,0.148478,0.028716,0.249434,0.262195,0.108560,0.100781,0.190106,0.116512,0.088653,0.076834,0.103985,0.072822,0.157246
2020-12-31,0.165877,0.183316,0.048410,0.132367,0.009087,0.198014,0.009180,0.264130,0.308092,0.037039,0.026449,0.197749,0.029494,0.180151,0.214523,0.247897
2021-12-31,0.182663,0.287287,0.201483,0.052131,0.000169,0.145727,0.156657,-0.000767,-0.001717,0.253109,0.240509,-0.025682,0.202416,0.017237,0.238201,0.287287
2022-12-31,-0.180034,-0.181754,-0.048812,-0.075137,0.007980,-0.233069,-0.162709,-0.053976,-0.053708,-0.168293,-0.242006,-0.154845,-0.056282,-0.045601,0.036681,-0.003787


In [112]:
ts = Tearsheet(port_price_returns[port_price_returns.index >= "2014-01-01"], "EW")
ts.summary(ann_factor=12).round(2)

,vt,spy,IVY,EW,RP,60_40,RAAB,VAAG12,VAAG4,GDM,DGDM,KDAAA,TIOF,GPM,HAA4,HAA1
#obs,141.00,141.00,141.00,141.00,141.00,141.00,141.00,141.00,141.00,141.00,141.00,141.00,141.00,141.00,141.00,141.00
#years,11.66,11.66,11.66,11.66,11.66,11.66,11.66,11.66,11.66,11.66,11.66,11.66,11.66,11.66,11.66,11.66
Total Return,1.96,3.35,0.82,1.20,0.26,1.74,1.14,0.74,1.11,1.40,1.01,0.51,0.59,1.00,1.71,3.49
Annual Return,0.10,0.13,0.05,0.07,0.02,0.09,0.07,0.05,0.07,0.08,0.06,0.04,0.04,0.06,0.09,0.14
Volatility,0.14,0.15,0.06,0.06,0.01,0.11,0.13,0.06,0.07,0.12,0.11,0.08,0.05,0.06,0.08,0.09
Max Drawdown,-0.26,-0.24,-0.06,-0.12,-0.01,-0.26,-0.24,-0.14,-0.10,-0.20,-0.25,-0.23,-0.08,-0.10,-0.07,-0.09
Skewness,-0.44,-0.41,-0.20,-0.01,0.52,-0.37,-0.60,0.48,0.56,-0.60,-0.58,-0.08,0.23,0.17,0.31,-0.02
Kurtosis,4.17,3.67,3.33,3.24,2.72,3.82,7.64,6.06,5.93,4.31,3.54,3.37,5.21,3.72,4.88,4.96
Sharpe Ratio,0.72,0.93,0.90,1.14,2.62,0.85,0.58,0.81,0.92,0.67,0.60,0.47,0.89,0.95,1.12,1.44
Standard Error,0.30,0.30,0.30,0.30,0.33,0.30,0.30,0.30,0.30,0.30,0.29,0.29,0.30,0.30,0.31,0.32


In [113]:
import numpy as np

capital = 100000  # initial capital

next_rebal = rebalance_dates.max()
print(f"Next rebalance date: {next_rebal.date()}")
# extract the weights at that date (MultiIndex: Date, ID) -> index of asset IDs
try:
    weights_next = all_strategies.xs(next_rebal, level=0).fillna(0)
except Exception:
    # fall back to .loc in case of different structure
    weights_next = all_strategies.loc[next_rebal].fillna(0)
    # Ensure we have a DataFrame with columns per strategy and index=asset IDs

if isinstance(weights_next, pd.DataFrame):
    for strat in weights_next.columns:
        print(f"Strategy: {strat}")
        s = weights_next[strat]
        s_nonzero = s[s != 0]
        if s_nonzero.empty:
            print(" No positions (all zeros)")
        else:
            for asset, w in s_nonzero.items():
                print(f" {asset}: {w*capital:.4f}")
        print("-" * 40)

Next rebalance date: 2025-09-30
Strategy: vt
 VT: 100000.0000
----------------------------------------
Strategy: spy
 SPY: 100000.0000
----------------------------------------
Strategy: IVY
 AGG: 20000.0000
 DBC: 20000.0000
 VEU: 20000.0000
 VNQ: 20000.0000
 VTI: 20000.0000
----------------------------------------
Strategy: EW
 BIL: 25000.0000
 BND: 25000.0000
 GLD: 25000.0000
 SPY: 25000.0000
----------------------------------------
Strategy: RP
 No positions (all zeros)
----------------------------------------
Strategy: 60_40
 SPY: 60000.0000
 TLT: 40000.0000
----------------------------------------
Strategy: RAAB
 BIL: 10526.3158
 DBC: 10526.3158
 EFA: 10526.3158
 EFV: 10526.3158
 IEF: 10526.3158
 VNQ: 47368.4211
----------------------------------------
Strategy: VAAG12
 EEM: 20000.0000
 EWJ: 20000.0000
 GLD: 20000.0000
 IWM: 20000.0000
 QQQ: 20000.0000
----------------------------------------
Strategy: VAAG4
 EEM: 100000.0000
----------------------------------------
Strategy: GDM
 